# Proper Integration tests

In [111]:
# If you have problems with import modules, uncomment line below and run it *ONE* time
# %cd ../

In [128]:
import requests
from data_generators.generators import RandomMentor, RandomMentee, RandomMenteeFeedback
from app.data import MongoDB

In [113]:
# API URL
url = "http://127.0.0.1:8000"

## Mentor Operations

In [146]:
# URL for the request

# Mentors
url_create_mentor = f'{url}/create/mentor'
url_read_mentor = f'{url}/read/mentor'
url_update_mentor = f'{url}/update/mentor/'

# Mentees
url_create_mentee = f'{url}/create/mentee'
url_read_mentee = f'{url}/read/mentee'
url_update_mentee = f'{url}/update/mentee/'

# Feedback
url_create_feedback = f'{url}/create/feedback'
url_read_feedback = f'{url}/read/feedback'
url_update_feedback = f'{url}/update/feedback'
url_delete_feedback = f'{url}/delete/feedback'

# Graph
url_tech_stack_by_role_graph = f'{url}/graph/tech-stack-by-role'


### Create Mentor
POST `/create/mentor`

### Setup for Mentors
Create a list of randomly generated mock mentor data and mark them as 'test_INTEGRATION_test' for post-testing cleanup.

In [140]:
# Set number of mentors to test, should be the same for mentor and mentees, because it is used late to create a feedbacks
n_persons = 3
# Create list of test mentors
mentors = []

for i in range(n_persons):
    mentors.append(vars(RandomMentor()))
    mentors[i]['other_info'] = 'test_INTEGRATION_test'


Use `/create/mentors/` endpoint to create a single mentor using properly formed mock data.
Verify that the status code returns 200 [successful], and that the expected JSON values are returned.

In [116]:
# Create a ONE test mentor with API, we should receive 'result': True
expected = {"result": True}
expected_status = 200
ret = requests.post(url_create_mentor, json=mentors[0])

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert expected == ret.json(), "Endpoint did not return expected value of {\"result\": True}"


### Read Mentor
POST `/read/mentor`

### Read one test mentor by querying `profile_id`

In [117]:
# Read first test mentor with unique profile_id
expected_status = 200
expected = 'test_INTEGRATION_test'

ret = requests.post(url_read_mentor, json={'profile_id': mentors[0]['profile_id']})

assert ret.json()['result'][0]['other_info'] == expected, f"Endpoint did not return expected value of {expected}"
assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"


### Read all entries
When we request read mentors without parameters we should get all entries, we added 1 mentor, so we assume that we should get more than 1 entries.

In [118]:
ret.json()

{'result': [{'profile_id': '4209092a-9879-4092-9602-935aca56928e',
   'first_name': 'Hendrix',
   'last_name': 'Ramirez',
   'email': 'Hendrix.Ramirez@gmail.com',
   'country': 'U.S.',
   'state': 'Utah',
   'city': 'Washington',
   'current_company': 'Alphabet',
   'current_position': 'Project Manager',
   'tech_stack': ['Android', 'Backend'],
   'job_help': False,
   'industry_knowledge': True,
   'pair_programming': False,
   'commitment': True,
   'referred_by': 'Meta',
   'other_info': 'test_INTEGRATION_test',
   'validate_status': 'pending',
   'is_active': True,
   'accepting_new_mentees': False,
   'created_at': '2022-10-11T20:46:53.222000'}]}

In [119]:
# read from API, with no parameteres, we should get all the data
expected_status = 200
ret = requests.post(url_read_mentor)

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert len(ret.json()['result']) > 1, f"We receive less that 2 entries. {len(ret.json()['result'])}"


### Update Mentors
POST `/update/mentor/{profile_id}`

Update existing test Mentor

In [120]:
# Update test Mentor current_company value
expected_status = 200

ret = requests.post(url_update_mentor  + mentors[0]['profile_id'], json={'current_company': 'test_Bloomtech_test'})

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"

# Read the query with our test value
expected_status = 200
expected = 'test_Bloomtech_test'

ret = requests.post(url_read_mentor, json={'current_company': 'test_Bloomtech_test'})

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert ret.json()['result'][0]['current_company'] == expected , f"Endpoint return something else we've expected"

## Mentee Operations



### Create Mentee
POST `/create/mentee`

### Generate a list of random *test* mentees

In [121]:
# Create list of test mentors
mentees = []

for i in range(n_persons):
    mentees.append(vars(RandomMentee()))

    # create a field to identify our test records later
    mentees[i]['other_info'] = 'test_INTEGRATION_test'


### Create *one* test mentee with API, status code should be 200


In [122]:
expected = {"result": True}
expected_status = 200

ret = requests.post(url_create_mentee, json=mentees[0])

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert expected == ret.json(), "Endpoint did not return expected value of {\"result\": True}"

### Read mentee
POST `/read/mentee`

### Read one test mentee by querying `profile_id`

In [123]:
# Read first test mentror with unique profile_id
expected_status = 200
expected = 'test_INTEGRATION_test'

ret = requests.post(url_read_mentee, json={'profile_id': mentees[0]['profile_id']})

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert ret.json()['result'][0]['other_info'] == expected, f"Endpoint did not return expected value of {expected}"

### Read all entrees


In [124]:
# read from API
expected_status = 200

ret = requests.post(url_read_mentee)

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert len(ret.json()['result']) > 1, f"We receive less that 2 entries. {len(ret.json()['result'])}"

### Update Mentees
POST `/update/mentee/{profile_id}`

### Update existing test Mentor

In [125]:
# Update test Mentor current_company value
expected_status = 200

ret = requests.post(url_update_mentee  + mentees[0]['profile_id'], json={'tech_stack': 'test_Bloomtech_test'})

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"

expected_status = 200
expected = 'test_Bloomtech_test'

# Read the query with our test value
ret = requests.post(url_read_mentee, json={'tech_stack': 'test_Bloomtech_test'})

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert expected == ret.json()['result'][0]['tech_stack'], f"Endpoint did not return expected value of {expected}"


## Feedback Operations


### Generate random feedback

In [144]:
feedbacks = []

for i in range(n_persons):
    feedbacks.append(vars(RandomMenteeFeedback(mentees[i]['profile_id'], mentors[i]['profile_id'])))
    feedbacks[i]['text'] = "test_INTEGRATION_test"

assert feedbacks[0]['ticket_id'] != '', f"Something wrong with RandomMenteeFeedback() generator"


### Create Feedback
POST `/create/feedback`

In [151]:
# Create ONE feedback
expected_status = 200
expected = {'result': True}

ret = requests.post(url_create_feedback, json=feedbacks[0])

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert ret.json() == expected, f"Endpoint did not return expected value of {expected}"

AssertionError: Return status code was not 200. Got status code 500

### Read Feedback
POST `/read/feedback`

### Update Feedback
PATCH `/update/feedback`

### Delete feedback
DELETE `/delete/feedback`

## Graph Operations

### Tech Stack By Role
GET `/graph/tech-stack-by-role`

# Cleaning
test data

In [126]:
db = MongoDB()

for collection in ['Mentors', 'Mentees']:
    db.delete(collection, {'other_info': 'test_INTEGRATION_test'})